# Importing necessary libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install mtcnn

In [ ]:
import cv2 as cv
import os
import numpy as np
import matplotlib.pyplot as plt
from mtcnn.mtcnn import MTCNN
from keras.preprocessing.image import ImageDataGenerator
from skimage import io
from PIL import Image
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
detector = MTCNN()

# Implementation

## Data Augmentation

In [ ]:
def face_augmentation(path):
    datagen = ImageDataGenerator(
            rotation_range = 15,
            shear_range = 0.2,
            zoom_range = 0.35,
            horizontal_flip = True,
            brightness_range = (0.5, 1.25))

    image_directory = path
    SIZE = 224
    dataset = []
    my_images = os.listdir(image_directory)
    for i, image_name in enumerate(my_images):
      print(image_name)
      if image_name.split('.')[-1] == 'jpg' or image_name.split('.')[-1] == 'jpeg':
          image = io.imread(image_directory + image_name)
          try:
            image = Image.fromarray(image, 'RGB')
            image = image.resize((SIZE,SIZE))
            image = np.array(image)
            #image = image.reshape((SIZE, SIZE, 1))
            dataset.append(np.array(image))
          except:
            image = Image.fromarray(image, 'L')
            image = image.resize((SIZE,SIZE))
            image = np.array(image)
            image = image.reshape((SIZE, SIZE, 1))
            dataset.append(np.array(image))

    x = np.array(dataset)
    i = 0
    for batch in datagen.flow(x, batch_size=16,
                              save_to_dir= path,
                              save_prefix='AU',
                              save_format='jpg'):
        i += 1
        if i > 4 :
            break

In [ ]:
train_path = '/content/drive/MyDrive/Database'

for d in os.listdir(train_path):
  p = train_path + '/' + d + '/'

joe.jpg


## Detection Phase

In [ ]:
class FaceDetection:
    def __init__(self, directory):
        self.directory = directory
        self.target_size = (160,160)
        self.X = []
        self.Y = []
        self.detector = MTCNN()

    def extract_face(self, filename):
        _image = cv.imread(filename)
        _image = cv.cvtColor(_image, cv.COLOR_BGR2RGB)
        node_x,node_y,node_w,node_h = self.detector.detect_faces(_image)[0]['box']
        node_x,node_y = abs(node_x), abs(node_y)
        face = _image[node_y:node_y+node_h, node_x:node_x+node_w]
        face_arr = cv.resize(face, self.target_size)
        return face_arr

    def load_faces(self, dir):
        extracted_faces = []
        for im_name in os.listdir(dir):
            try:
                path = dir + im_name
                single_face = self.extract_face(path)
                extracted_faces.append(single_face)
            except:
                print("couldn't detect")
                pass
        return extracted_faces

    def load_classes(self):
        for sub_dir in os.listdir(self.directory):
            path = self.directory +'/'+ sub_dir+'/'
            FACES = self.load_faces(path)
            print(path)
            labels = [sub_dir for _ in range(len(FACES))]
            print(f"Loaded successfully: {len(labels)}")
            self.X.extend(FACES)
            self.Y.extend(labels)
        return np.asarray(self.X), np.asarray(self.Y)

    def plot_images(self):
        plt.figure(figsize=(18,16))
        for _index,face in enumerate(self.X):
            cols = 3
            rows = len(self.Y) // cols + 1
            plt.subplot(rows, cols, _index + 1)
            plt.imshow(face)
            plt.axis('off')

In [ ]:
faces = FaceDetection(r"/content/drive/MyDrive/Database")
X, Y = faces.load_classes()

In [ ]:
plt.figure(figsize=(18,18))
for n,image in enumerate(X):
    num_cols = 6
    num_rows = len(Y) // num_cols + 1
    plt.subplot(num_rows, num_cols, n + 1)
    plt.title(f'{Y[n]}')
    plt.imshow(image)
    plt.axis('off')

## Feature Extraction

In [ ]:
!pip install keras_facenet

In [ ]:
from keras_facenet import FaceNet
embedder = FaceNet()

def get_embedding(face_image):
  face_img = face_image.astype('float32') # 3D(160x160x3)
  face_img = np.expand_dims(face_img, axis=0) 
  # 4D (None x 160x160x3)
  yhat= embedder.embeddings(face_img)
  return yhat[0] # 512D image (1x1x512)

In [ ]:
EMBEDDED_X = []

for img in X:
    EMBEDDED_X.append(get_embedding(img))

EMBEDDED_X = np.asarray(EMBEDDED_X)

In [ ]:
np.savez_compressed('face_embeddings.npz', EMBEDDED_X, Y)

## Classification

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)

# table for no encoder and svm on embeddings
Y

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(EMBEDDED_X, Y, shuffle=True, random_state=17)

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='linear', probability=True)
svm.fit(X_train, Y_train)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(criterion='gini')
decision_tree.fit(X_train, Y_train)

In [ ]:
decision_tree_test = decision_tree.predict(X_test)
decision_tree_train = decision_tree.predict(X_train)

from sklearn.metrics import accuracy_score
accuracy_score(Y_train, decision_tree_train) * 100, accuracy_score(Y_test, decision_tree_test) * 100

In [ ]:
svm_train = svm.predict(X_train)
svm_test = svm.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_score(Y_train, svm_train) * 100, accuracy_score(Y_test, svm_test) * 100

In [ ]:
from sklearn.metrics import confusion_matrix
 
cm = confusion_matrix(Y_test,decision_tree_test)
cm

In [ ]:
import plotly.express as px
px.imshow(cm, labels=dict(x="Prediction", y="Truth"), color_continuous_scale="Inferno", text_auto=True)

In [ ]:
import plotly.express as px
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc

y_score = svm.predict_proba(X_test)[:, 1]

fpr, tpr, thresholds = roc_curve(Y_train, y_score)

fig = px.area(
    x=fpr, y=tpr,
    title=f'ROC Curve (AUC={auc(fpr, tpr):.4f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.show()

## Testing

In [ ]:
def svm_test(directory):
    res = []
    nms = []
    actual = []
    correct = 0
    for sub_dir in os.listdir(directory):
        path = directory + '/' + sub_dir + '/'
        actual.append(sub_dir)
        for im_name in os.listdir(path):
            f_path = path +'/'+ im_name
            person = cv.imread(f_path)
            print(im_name)
            person = cv.cvtColor(person, cv.COLOR_BGR2RGB)
            try:
                node__x,node__y,node__w,node__h = detector.detect_faces(person)[0]['box']
                face = person[node__y:node__y + node__h, node__x:node__x + node__w]
                face = cv.resize(face, (160,160))
                test_emb = get_embedding(face)
                f_pred = svm.predict([test_emb])
                res.append(person)
                nms.append(encoder.inverse_transform(f_pred)[0])
                if sub_dir == encoder.inverse_transform(f_pred)[0]:
                    correct += 1
            except:
                pass
    return res, nms, actual, correct

def decision_tree_test(directory):
    res = []
    nms = []
    actual = []
    correct = 0
    for sub_dir in os.listdir(directory):
        path = directory + '/' + sub_dir + '/'
        actual.append(sub_dir)
        for im_name in os.listdir(path):
            f_path = path +'/'+ im_name
            person = cv.imread(f_path)
            print(im_name)
            person = cv.cvtColor(person, cv.COLOR_BGR2RGB)
            try:
                node__x,node__y,node__w,node__h = detector.detect_faces(person)[0]['box']
                face = person[node__y:node__y + node__h, node__x:node__x + node__w]
                face = cv.resize(face, (160,160))
                test_emb = get_embedding(face)
                f_pred = svm.predict([test_emb])
                res.append(person)
                nms.append(encoder.inverse_transform(f_pred)[0])
                if sub_dir == encoder.inverse_transform(f_pred)[0]:
                    correct += 1
            except:
                pass
    return res, nms, actual, correct

In [ ]:
%%time
svm_im = svm_test(r"/content/drive/MyDrive/Datasets (1)/JAFFE Dataset/val")

In [ ]:
%%time
decision_tree_im = decision_tree_test(r"/content/drive/MyDrive/Datasets (1)/JAFFE Dataset/val")

In [ ]:
svm_people, svm_names, svm_images, svm_corrects = svm_im[2], svm_im[1], svm_im[0], svm_im[3]
[svm_corrects, len(svm_names), svm_corrects/len(svm_names)]

In [ ]:
decision_tree_people, decision_tree_names, decision_tree_images, decision_tree_corrects = decision_tree_im[2], decision_tree_im[1], decision_tree_im[0], decision_tree_im[3]
[decision_tree_corrects, len(decision_tree_names), decision_tree_corrects/len(decision_tree_names)]

In [ ]:
fig = plt.figure(figsize= (18, 18))
for i, img in enumerate(decision_tree_images):
  ax = fig.add_subplot(12, 5, i+1)
  plt.title(f'{decision_tree_names[i]}')
  plt.axis('off')
  ax.imshow(img)

In [ ]:
import pickle
#save the model
with open('students_model_v1.pkl','wb') as f:
    pickle.dump(svm,f)


In [ ]:
import cv2
import matplotlib.pyplot as plt

file = '/content/drive/MyDrive/Database/JOE/' 
ims =[]
for im_name in os.listdir(file):
  f_path = file +'/'+ im_name
  person = cv.imread(f_path)
  person = cv.cvtColor(person, cv.COLOR_BGR2RGB)
  ims.append(person)

fig = plt.figure(figsize= (18, 18))
for i, img in enumerate(ims):
  ax = fig.add_subplot(1, 6, i+1)
  plt.axis('off')
  ax.imshow(img)